### **Installing libraries that is requirred for the project**

In [1]:
!pip install -U -q "google-generativeai>=0.8.2"

## **Importing necessary modules or libraries**

In [2]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:

    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai





# **Loading the model and pass the arguements of the model**

In [3]:
# Parse the arguments

model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6ImVzc2F5IG9uIGFpIn1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiIyMgVGhlIEV2b2x2aW5nIExhbmRzY2FwZSBvZiBBcnRpZmljaWFsIEludGVsbGlnZW5jZTogQSBTeW1waG9ueSBvZiBQcm9ncmVzcyBhbmQgVW5jZXJ0YWludHlcXG5cXG5BcnRpZmljaWFsIGludGVsbGlnZW5jZSAoQUkpIGhhcyB0cmFuc2l0aW9uZWQgZnJvbSBhIHNjaWVuY2UgZmljdGlvbiBjb25jZXB0IHRvIGEgcGVydmFzaXZlIGZvcmNlIGluIG91ciBtb2Rlcm4gd29ybGQuICBJdCBpcyByZXNoYXBpbmcgaW5kdXN0cmllcywgYXV0b21hdGluZyB0YXNrcywgYW5kIGV2ZW4gaW5mbHVlbmNpbmcgb3VyIHBlcnNvbmFsIGludGVyYWN0aW9ucy4gIFRoaXMgZXNzYXkgd2lsbCBkZWx2ZSBpbnRvIHRoZSBtdWx0aWZhY2V0ZWQgbmF0dXJlIG9mIEFJLCBleHBsb3JpbmcgaXRzIHBvdGVudGlhbCBiZW5lZml0cywgZXRoaWNhbCBjb25zaWRlcmF0aW9ucywgYW5kIHRoZSBwcm9mb3VuZCBpbXBhY3QgaXQgaGFzIG9uIGh1bWFuaXR5J3MgZnV0dXJlLlxcblxcbkF0IHRoZSBjb3JlIG9mIEFJIGxpZXMgdGhlIGFzcGlyYXRpb24gdG8gbWltaWMgaHVtYW4gaW50ZWxsaWdlbmNlLiAgVGhyb3VnaCBzb3BoaXN0aWNhdGVkIGFsZ29yaXRobXMgYW5kIG1hY2hpbmUgbGVhcm5pbmcsIEFJIHN5c3RlbXMgYXJlIHRyYWluZWQgdG8gYW5hbHl6ZSB2YXN0IGFtb3VudHMgb2YgZGF0YSwgcmVjb2duaXplIHBhdHRlcm5zLCBhbmQgbWFrZSBwcmVkaWN0aW9ucy4gIFRoaXMgYWJpbGl0eSBoYXMgbGVkIHRvIHJlbWFya2FibGUgYWR2YW5jZW1lbnRzIGluIHZhcmlvdXMgZmllbGRzLiBJbiBoZWFsdGhjYXJlLCBBSSBhc3Npc3RzIGluIGRpYWdub3NpbmcgZGlzZWFzZXMsIGRldmVsb3BpbmcgcGVyc29uYWxpemVkIHRyZWF0bWVudCBwbGFucywgYW5kIGV2ZW4gYWNjZWxlcmF0aW5nIGRydWcgZGlzY292ZXJ5LiAgSW4gZmluYW5jZSwgaXQgZW1wb3dlcnMgYWxnb3JpdGhtcyB0byBhbmFseXplIG1hcmtldCB0cmVuZHMsIG1hbmFnZSByaXNrLCBhbmQgb3B0aW1pemUgaW52ZXN0bWVudCBzdHJhdGVnaWVzLiAgVGhlIGxpc3Qgb2YgYXBwbGljYXRpb25zIGdvZXMgb24sIGZyb20gc2VsZi1kcml2aW5nIGNhcnMgdG8gcGVyc29uYWxpemVkIHZpcnR1YWwgYXNzaXN0YW50cywgZGVtb25zdHJhdGluZyB0aGUgd2lkZS1yZWFjaGluZyBpbXBhY3Qgb2YgQUkuXFxuXFxuSG93ZXZlciwgdGhlIHJpc2Ugb2YgQUkgYWxzbyBicmluZ3Mgd2l0aCBpdCBhIGNvbXBsZXggd2ViIG9mIGV0aGljYWwgY29uY2VybnMuICBPbmUgY3J1Y2lhbCBhcmVhIGlzIGJpYXMuICBBSSBzeXN0ZW1zLCB0cmFpbmVkIG9uIGhpc3RvcmljYWwgZGF0YSwgbWF5IGluYWR2ZXJ0ZW50bHkgaW5oZXJpdCBhbmQgYW1wbGlmeSBleGlzdGluZyBiaWFzZXMsIGxlYWRpbmcgdG8gdW5mYWlyIG9yIGRpc2NyaW1pbmF0b3J5IG91dGNvbWVzLiAgVGhpcyByYWlzZXMgY3JpdGljYWwgcXVlc3Rpb25zIGFib3V0IGFjY291bnRhYmlsaXR5LCB0cmFuc3BhcmVuY3ksIGFuZCB0aGUgbmVlZCBmb3Igcm9idXN0IGV0aGljYWwgZnJhbWV3b3JrcyB0byBlbnN1cmUgZmFpcm5lc3MgYW5kIGVxdWl0YWJsZSBhcHBsaWNhdGlvbiBvZiBBSS5cXG5cXG5Bbm90aGVyIHNpZ25pZmljYW50IGNvbmNlcm4gaXMgdGhlIHBvdGVudGlhbCBmb3Igam9iIGRpc3BsYWNlbWVudC4gIEFzIEFJIGF1dG9tYXRlcyB0YXNrcyBwcmV2aW91c2x5IHBlcmZvcm1lZCBieSBodW1hbnMsIGNvbmNlcm5zIGFib3V0IHVuZW1wbG95bWVudCBhbmQgc29jaWV0YWwgZGlzcnVwdGlvbiBhcmlzZS4gIFRoaXMgZGVtYW5kcyBhIHByb2FjdGl2ZSBhcHByb2FjaCB0byByZXNraWxsaW5nIGFuZCB1cHNraWxsaW5nIHRoZSB3b3JrZm9yY2UsIGVuc3VyaW5nIHRoYXQgaW5kaXZpZHVhbHMgY2FuIGFkYXB0IHRvIHRoZSBldm9sdmluZyBqb2IgbWFya2V0IGFuZCBlbWJyYWNlIG9wcG9ydHVuaXRpZXMgY3JlYXRlZCBieSBBSS5cXG5cXG5GdXJ0aGVybW9yZSwgdGhlIHJhcGlkIGFkdmFuY2VtZW50IG9mIEFJIHJhaXNlcyBxdWVzdGlvbnMgYWJvdXQgaXRzIHBvdGVudGlhbCBpbmZsdWVuY2Ugb24gaHVtYW4gYXV0b25vbXkuICBUaGUgcmVsaWFuY2Ugb24gQUkgZm9yIGRlY2lzaW9uLW1ha2luZywgcGFydGljdWxhcmx5IGluIGNyaXRpY2FsIGFyZWFzIGxpa2UgaGVhbHRoY2FyZSBhbmQgbGF3IGVuZm9yY2VtZW50LCByYWlzZXMgY29uY2VybnMgYWJvdXQgdGhlIGVyb3Npb24gb2YgaHVtYW4gYWdlbmN5IGFuZCB0aGUgbmVlZCB0byBtYWludGFpbiBjb250cm9sIG92ZXIgb3VyIG93biBkZXN0aW5pZXMuICBUaGlzIGNhbGxzIGZvciBvbmdvaW5nIGRpYWxvZ3VlIGFuZCBkZWxpYmVyYXRpb24gcmVnYXJkaW5nIHRoZSBib3VuZGFyaWVzIG9mIEFJIGFuZCBpdHMgaW50ZWdyYXRpb24gaW50byBodW1hbiBsaWZlLlxcblxcblRoZSBmdXR1cmUgb2YgQUkgcmVtYWlucyBhIHRhcGVzdHJ5IHdvdmVuIHdpdGggYm90aCBwcm9taXNlIGFuZCB1bmNlcnRhaW50eS4gIFdoaWxlIGl0cyBwb3RlbnRpYWwgdG8gZW5oYW5jZSBvdXIgbGl2ZXMsIHNvbHZlIGNvbXBsZXggcHJvYmxlbXMsIGFuZCBwdXNoIHRoZSBib3VuZGFyaWVzIG9mIGh1bWFuIGtub3dsZWRnZSBpcyBpbW1lbnNlLCBpdCBpcyBpbXBlcmF0aXZlIHRvIGFkZHJlc3MgaXRzIGV0aGljYWwgYW5kIHNvY2lldGFsIGltcGxpY2F0aW9ucy4gIEJ5IGZvc3RlcmluZyBvcGVuIGRpYWxvZ3VlLCBwcm9tb3RpbmcgcmVzcG9uc2libGUgZGV2ZWxvcG1lbnQsIGFuZCBlc3RhYmxpc2hpbmcgZXRoaWNhbCBndWlkZWxpbmVzLCB3ZSBjYW4gaGFybmVzcyB0aGUgdHJhbnNmb3JtYXRpdmUgcG93ZXIgb2YgQUkgd2hpbGUgc2FmZWd1YXJkaW5nIG91ciB2YWx1ZXMgYW5kIGVuc3VyaW5nIGEgZnV0dXJlIHdoZXJlIHRlY2hub2xvZ3kgc2VydmVzIGh1bWFuaXR5LiBcXG5cXG5VbHRpbWF0ZWx5LCBBSSBpcyBub3QgYSB0aHJlYXQsIGJ1dCByYXRoZXIgYSBwb3dlcmZ1bCB0b29sLiAgSXQgaXMgb3VyIHJlc3BvbnNpYmlsaXR5IHRvIHdpZWxkIGl0IHdpdGggd2lzZG9tLCBjb21wYXNzaW9uLCBhbmQgYSBkZWVwIHVuZGVyc3RhbmRpbmcgb2YgaXRzIGltcGFjdCBvbiBvdXIgc29jaWV0eSwgc2hhcGluZyBhIGZ1dHVyZSB3aGVyZSBBSSBlbXBvd2VycyB1cyB0byByZWFjaCBvdXIgZnVsbCBwb3RlbnRpYWwgYXMgaHVtYW4gYmVpbmdzLiBcXG4ifV19XQ==' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# **Converting input prompt into files and pass them into the model**

In [4]:
# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "role": "user",
        "parts": [
            {
                "text": "essay on ai"
            }
        ]
    },
    {
        "role": "model",
        "parts": [
            {
                "text": "## The Evolving Landscape of Artificial Intelligence: A Symphony of Progress and Uncertainty\\n\\nArtificial intelligence (AI) has transitioned from a science fiction concept to a pervasive force in our modern world.  It is reshaping industries, automating tasks, and even influencing our personal interactions.  This essay will delve into the multifaceted nature of AI, exploring its potential benefits, ethical considerations, and the profound impact it has on humanity's future.\\n\\nAt the core of AI lies the aspiration to mimic human intelligence.  Through sophisticated algorithms and machine learning, AI systems are trained to analyze vast amounts of data, recognize patterns, and make predictions.  This ability has led to remarkable advancements in various fields. I

# **Defining a function to call the model**

In [5]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
def prompt(model,prompt):
  gemini = genai.GenerativeModel(model_name=model)

  response = gemini.generate_content("provide suitable legal action"+prompt,
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=stream,
  )
  display(Markdown(response.text))

Overwriting app.py


34.83.234.246





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.234.246:8501

your url is: https://bitter-walls-trade.loca.lt
/root/.npm/_npx/75ac80b86e83d4a2/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:22459 (check your firewall settings)
    at Socket.<anonymous> (/root/.npm/_npx/75ac80b86e83d4a2/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:513:28)
    at emitErrorNT (node:internal/streams/destroy:157:8)
    at emitErrorCloseNT (node:internal/streams/destroy:122:3)
    at processTicksAndRejections (node:internal/process/task_queues:83:21)
  Stopping...
